In [1]:
using Pkg

Pkg.add("CuArrays")

Pkg.add("CUDAnative")

Pkg.add("CUDAdrv")

using CUDAnative, CUDAdrv, CuArrays

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
  [be33ccc6] + CUDAnative v2.1.3
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
  [c5f51814] + CUDAdrv v3.0.1
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


Internal error: encountered unexpected error in runtime:
BoundsError(a=Array{Core.Compiler.BasicBlock, (32,)}[
  Core.Compiler.BasicBlock(stmts=Core.Compiler.StmtRange(first=1, last=7), preds=Array{Int64, (1,)}[32], succs=Array{Int64, (1,)}[2]),
  Core.Compiler.BasicBlock(stmts=Core.Compiler.StmtRange(first=8, last=14), preds=Array{Int64, (1,)}[1], succs=Array{Int64, (2,)}[5, 3]),
  Core.Compiler.BasicBlock(stmts=Core.Compiler.StmtRange(first=15, last=20), preds=Array{Int64, (1,)}[2], succs=Array{Int64, (1,)}[4]),
  Core.Compiler.BasicBlock(stmts=Core.Compiler.StmtRange(first=21, last=21), preds=Array{Int64, (1,)}[3], succs=Array{Int64, (1,)}[7]),
  Core.Compiler.BasicBlock(stmts=Core.Compiler.StmtRange(first=22, last=28), preds=Array{Int64, (1,)}[2], succs=Array{Int64, (1,)}[6]),
  Core.Compiler.BasicBlock(stmts=Core.Compiler.StmtRange(first=29, last=29), preds=Array{Int64, (1,)}[5], succs=Array{Int64, (1,)}[7]),
  Core.Compiler.BasicBlock(stmts=Core.Compiler.StmtRange(first=30, last=

In [2]:
Pkg.add("BenchmarkTools")

Pkg.add("Test")

using BenchmarkTools, Test

 Resolving package versions...
 Installed BenchmarkTools ─ v0.4.2
  Updating `~/.julia/environments/v1.0/Project.toml`
  [6e4b80f9] + BenchmarkTools v0.4.2
  Updating `~/.julia/environments/v1.0/Manifest.toml`
  [6e4b80f9] + BenchmarkTools v0.4.2
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
  [8dfed614] + Test 
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


┌ Info: Precompiling BenchmarkTools [6e4b80f9-dd63-53aa-95a3-0cdb28fa8baf]
└ @ Base loading.jl:1192


In [22]:
#Initializing necessary matrices and variables

N=parse(Int64,readline())

M1=rand(Float32,N,N)

M2=rand(Float32,N,N)

ParallelM=zeros(Float32,N,N)

SerialM=zeros(Float32,N,N);


stdin> 2000


In [23]:
function sequential_multiplication!(M1,M2,SerialM,N)
 for i in 1:N
  for j in 1:N
   sum = 0
   for k in 1:N
    sum += M1[i,k] * M2[k,j]
   end
   SerialM[i,j] = sum
  end
 end
end 
 

sequential_multiplication! (generic function with 1 method)

In [24]:
#Transferring data to the device from the host

D_ParallelM=CuArray(ParallelM)

D_M1=CuArray(M1)

D_M2=CuArray(M2)


2000×2000 CuArray{Float32,2}:
 0.602119   0.910059   0.256858   …  0.241932   0.395225    0.717418  
 0.203087   0.689398   0.0377104     0.686151   0.00226462  0.840313  
 0.523306   0.302685   0.877051      0.687856   0.590213    0.832803  
 0.407511   0.688244   0.897145      0.183727   0.881352    0.160002  
 0.838856   0.0140892  0.831357      0.509102   0.847047    0.481481  
 0.330664   0.437633   0.307116   …  0.12902    0.0938993   0.622898  
 0.57943    0.708338   0.487313      0.279438   0.277556    0.0635089 
 0.431431   0.740841   0.912193      0.551823   0.700631    0.97049   
 0.248804   0.0759093  0.203614      0.197023   0.504779    0.38533   
 0.697608   0.576224   0.99373       0.100421   0.436154    0.0122278 
 0.703544   0.373988   0.228922   …  0.606856   0.912959    0.0589983 
 0.6105     0.993407   0.431615      0.670306   0.293111    0.00117838
 0.509377   0.485065   0.131433      0.455341   0.281696    0.424354  
 ⋮                                ⋱            

In [25]:
#Matrix Multiplication in the Parallel Format

function parallel_multiplication!(M1,M2,ParallelM,N)
 x = (blockIdx().x-1)*blockDim().x + threadIdx().x 
 y = (blockIdx().y-1)*blockDim().y + threadIdx().y 
 if(x<=N && y<=N)
  sum=0
  for i in 1:N
   @inbounds sum = sum + M1[x,i] * M2[i,y]
  end
 @inbounds ParallelM[x,y]=sum
 end 
return
end

parallel_multiplication! (generic function with 1 method)

In [26]:
@btime sequential_multiplication!(M1,M2,SerialM,N)
@btime @cuda blocks = (ceil(Int32,(N/32)),ceil(Int32,(N/32))) threads = (32,32) parallel_multiplication!(D_M1,D_M2,D_ParallelM,N)
H_ParallelM=Array(D_ParallelM);
@test isapprox(SerialM,H_ParallelM)

  48.169 s (0 allocations: 0 bytes)
  10.829 μs (69 allocations: 2.45 KiB)


Test Passed